In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5ff17c6f54fd23191499c7017d6366538fcc35b61d0225390f78827c662577ddcc8d5d64ddf58c68c56d6ad68e5721930a5b7a5803b0bee173'

In [2]:
import aocd
from aocd.models import Puzzle
day = 12
year = 2023
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [222]:
import re
from itertools import permutations, combinations
from functools import cache

In [4]:
test_data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
"""
data_test = test_data.splitlines()

In [5]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(1000,
 ['???????????#??? 5,2,1',
  '#??##.??.?# 5,1,1',
  '.??###?..???#?.?##?. 4,4,3',
  '??..?#??#??#?.???? 5,1,1',
  '?????#???? 2,1,1',
  '?#?##???#????? 1,8',
  '??#?.?#????.? 1,4',
  '?#####??#?????.?#.. 9,2,1',
  '??????#???? 1,3,1',
  '.?.?#?????#?.??.? 2,2'])

In [236]:
def parse(line):
    conditions, sums = line.split(" ")
    blocks = tuple([int(val) for val in sums.split(',')])
    return conditions, blocks

def gen_blocks(condition):
    blocks = []
    current = 0
    for c in condition:
        if c == '#':
            current += 1
        elif c == '.':
            if current:
                blocks.append(current)
                current = 0
    if current:
        blocks.append(current)
    return tuple(blocks)

def get_permutations(num_bad, total_len):
    for comb in combinations(range(total_len), num_bad):
        missing = ['.'] * total_len
        for c in comb:
            missing[c] = '#'
        yield missing
    

def gen_conditions(condition, num_bad):
    char = {0: '.', 1: '#'}
    n = condition.count('?')
    condition = condition.replace('?', '{}')
    # if n == 0:
    #     yield condition

    # missing = ['#'] * num_bad + ['.'] * (n - num_bad)
    # for perm in set(permutations(missing)):
    for missing in get_permutations(num_bad, n):
        yield condition.format(*missing)
        
def count_possible(spring):
    condition, blocks = spring
    # print(condition, blocks)
    num_bad = sum(blocks) - condition.count('#')
    possible = 0
    for cond in gen_conditions(condition, num_bad):
        # print(cond)
        if gen_blocks(cond) == blocks:
            possible += 1
    return possible


def unfold(spring):
    condition, blocks = spring
    condition = condition + '?' + condition + '?' + condition + '?' + condition + '?' + condition
    blocks = blocks * 5
    return condition, blocks

def count_possible_b(spring):
    condition, blocks = spring
    # print(condition, blocks)
    num_bad = sum(blocks) - condition.count('#')
    possible = 0
    for cond in gen_conditions(condition, num_bad):
        # print(cond)
        if gen_blocks(cond) == blocks:
            possible += 1
    return possible

def find_blocks(condition):
    blocks = []
    cur = 0
    
@cache
def count_rec(condition, blocks):    
    idx = condition.find('?')
    if idx == -1:
        if gen_blocks(condition) == blocks:
            return 1
        return 0
    # prune impossible paths
    if idx > 2:
        sure_blocks = gen_blocks(condition[:idx])
        lb = len(sure_blocks)
        if len(sure_blocks) > len(blocks):
            return 0
        if lb > 1:
            if condition[idx - 1] == '.':
                if sure_blocks != blocks[:lb]:
                    return 0
            elif (sure_blocks[:lb-1] != blocks[:lb-1]) or (sure_blocks[lb-1] > blocks[lb-1]):
                return 0
    
    condition_left = condition[:idx] + '.' + condition[idx+1:]
    condition_right = condition[:idx] + '#' + condition[idx+1:]
    # print(condition_left, condition_right)
    # if idx > 10:        
    #     print(idx, condition, blocks, sure_blocks, lb)
    #     return
    return count_rec(condition_left, blocks) + count_rec(condition_right, blocks)

@cache
def place_blocks_rec(condition, blocks):
    if not blocks:
        return '#' not in condition
    max_pos = len(condition) - (sum(blocks) + len(blocks) - 1)
    num_possible = 0
    bl = blocks[0]
    for pos in range(max_pos+1):
        if condition[pos] == '.':
            continue
        elif '.' not in condition[pos:pos+bl] and ((len(condition) == pos+bl ) or (condition[pos+bl] != '#')):
            num_possible += place_blocks_rec(condition[pos+bl+1:], blocks[1:])
        if condition[pos] == '#':
            break
    return num_possible


# def place_blocks_rec(condition, blocks):
#     if len(blocks) < 3:
#         # print(condition, blocks)
#         return count_rec(condition, blocks)
#     max_pos = len(condition) - (sum(blocks) + len(blocks) - 1)
#     num_possible = 0
#     bl = blocks[0]
#     for pos in range(max_pos+1):
#         if condition[pos] == '.':
#             continue
#         elif '.' not in condition[pos:pos+bl] and ((len(condition) == pos+bl ) or (condition[pos+bl] != '#')):
#             num_possible += place_blocks_rec(condition[pos+bl+1:], blocks[1:])
#         if condition[pos] == '#':
#             break
#     return num_possible

In [189]:
# springs = [parse(line) for line in data_test]
springs = [parse(line) for line in data]

In [190]:
%time
num_pos = [count_possible(spring) for spring in springs]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [191]:
sum(num_pos)

7204

In [193]:
num_pos = [place_blocks_rec(*spring) for spring in springs]
sum(num_pos)

7204

In [105]:
num_pos = [count_rec(*spring) for spring in springs]

In [32]:
puzzle.answer_a = sum(num_pos)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [194]:
# Part 2

In [237]:
springs = [parse(line) for line in data]

num_pos = 0
for i, spring in enumerate(springs):
    if ((i % 20) == 0):
        print(i)
    print('.', end="")
    num_pos += place_blocks_rec(*unfold(spring))

0
....................20
....................40
....................60
....................80
....................100
....................120
....................140
....................160
....................180
....................200
....................220
....................240
....................260
....................280
....................300
....................320
....................340
....................360
....................380
....................400
....................420
....................440
....................460
....................480
....................500
....................520
....................540
....................560
....................580
....................600
....................620
....................640
....................660
....................680
....................700
....................720
....................740
....................760
....................780
....................800
....................820
..................

In [238]:
num_pos

1672318386674

In [228]:
puzzle.answer_b = num_pos

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [235]:
place_blocks_rec(*unfold(springs[0]))

459591913